In [3]:

from transformers import AutoModelForCausalLM
import torch


model = AutoModelForCausalLM.from_pretrained("../models/gemma-3-270m-it_2912_v6/checkpoint-155_merged_fp32")


### Save LMHead + norm (6 different files)

In [ ]:
import math
import os
import torch
import numpy as np
from tqdm import tqdm
model._modules["model"]

norm = model._modules["model"].norm

if not os.path.exists("params"):
    os.makedirs("params")
with open(f"params/norm.bin", "wb") as f:
    f.write(norm.weight.detach().cpu().numpy().astype(np.float32).tobytes())


lm_head = model._modules["lm_head"].weight.detach()
lm_head_flat = lm_head.flatten()

os.makedirs("params/lm_head", exist_ok=True)

num_chunks = 14
chunks = torch.chunk(lm_head_flat, num_chunks)

for idx, chunk in enumerate(chunks):
    # Convert to float32 numpy array
    np_chunk = chunk.cpu().float().numpy()
    
    # Write raw binary
    with open(f"params/lm_head/part_{idx}.bin", "wb") as f:
        f.write(np_chunk.tobytes())
    
    print(f"Saved chunk {idx} with {np_chunk.size} weights")

Saved chunk 0 with 11983726 weights
Saved chunk 1 with 11983726 weights
Saved chunk 2 with 11983726 weights
Saved chunk 3 with 11983726 weights
Saved chunk 4 with 11983726 weights
Saved chunk 5 with 11983726 weights
Saved chunk 6 with 11983726 weights
Saved chunk 7 with 11983726 weights
Saved chunk 8 with 11983726 weights
Saved chunk 9 with 11983726 weights
Saved chunk 10 with 11983726 weights
Saved chunk 11 with 11983726 weights
Saved chunk 12 with 11983726 weights
Saved chunk 13 with 11983722 weights


### Save layers modules


In [ ]:
import os
import numpy as np
from tqdm import tqdm
for idx, layer in tqdm(enumerate(model._modules["model"].layers)):
    self_attn = layer.self_attn
    mlp = layer.mlp
    input_layernorm = layer.input_layernorm
    post_attention_layernorm = layer.post_attention_layernorm
    pre_feedforward_layernorm = layer.pre_feedforward_layernorm
    post_feedforward_layernorm = layer.post_feedforward_layernorm
    
    os.makedirs(f"params/layer_{idx}", exist_ok = True)
    
    # ================================================================ GQA =====================================================
    with open(f"params/layer_{idx}/self_attn_q_proj.bin", "wb")as f:
        f.write(self_attn.q_proj.weight.detach().cpu().float().flatten().numpy().tobytes())
        
    with open(f"params/layer_{idx}/self_attn_k_proj.bin", "wb")as f:
        f.write(self_attn.k_proj.weight.detach().cpu().float().flatten().numpy().tobytes())
        
    with open(f"params/layer_{idx}/self_attn_v_proj.bin", "wb")as f:
        f.write(self_attn.v_proj.weight.detach().cpu().float().flatten().numpy().tobytes())
    with open(f"params/layer_{idx}/self_attn_o_proj.bin", "wb")as f:
        f.write(self_attn.o_proj.weight.detach().cpu().float().flatten().numpy().tobytes())
        
    with open(f"params/layer_{idx}/self_attn_q_norm.bin", "wb")as f:
        f.write(self_attn.q_norm.weight.detach().cpu().float().flatten().numpy().tobytes())
    with open(f"params/layer_{idx}/self_attn_k_norm.bin", "wb")as f:
        f.write(self_attn.k_norm.weight.detach().cpu().float().flatten().numpy().tobytes())
        
    
    # =============================================================== MLP ======================================================
    with open(f"params/layer_{idx}/mlp_gate_proj.bin", "wb")as f:
        f.write(mlp.gate_proj.weight.detach().cpu().float().flatten().numpy().tobytes())
    with open(f"params/layer_{idx}/mlp_up_proj.bin", "wb")as f:
        f.write(mlp.up_proj.weight.detach().cpu().float().flatten().numpy().tobytes())
    with open(f"params/layer_{idx}/mlp_down_proj.bin", "wb")as f:
        f.write(mlp.down_proj.weight.detach().cpu().float().flatten().numpy().tobytes())
        
    # ================================================================ RMS =====================================================
    with open(f"params/layer_{idx}/input_layernorm.bin", "wb") as f:
        f.write(input_layernorm.weight.detach().cpu().float().flatten().numpy().tobytes())
    with open(f"params/layer_{idx}/post_attention_layernorm.bin", "wb")as f:
        f.write(post_attention_layernorm.weight.detach().cpu().float().flatten().numpy().tobytes())
    with open(f"params/layer_{idx}/pre_feedforward_layernorm.bin", "wb")as f:
        f.write(pre_feedforward_layernorm.weight.detach().cpu().float().flatten().numpy().tobytes())
    with open(f"params/layer_{idx}/post_feedforward_layernorm.bin", "wb")as f:
        f.write(post_feedforward_layernorm.weight.detach().cpu().float().flatten().numpy().tobytes())
        
    # print(vars(layer))

18it [00:04,  3.99it/s]
